In [1]:
import pandas as pd
import os
import datetime
import gc
import numpy as np
import glob
samplerows=None
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/Bruce_2019Q2_test2_test3_YoY'

In [2]:
first_weekend_2019_before=datetime.date(2019,5,11)
week_2019_before=[first_weekend_2019_before+datetime.timedelta(days=x*7) for x in range(6)]
week_2019_during=[first_weekend_2019_before+datetime.timedelta(days=(x+6)*7) for x in range(6)]
week_2018_before=[x-datetime.timedelta(days=52*7) for x in week_2019_before]
week_2018_during=[x-datetime.timedelta(days=52*7) for x in week_2019_during]

df_all_week_end_dts=pd.DataFrame()

df_2019_before=pd.DataFrame({"week_end_dts":week_2019_before,"groups":"2019_before"})
df_2019_during=pd.DataFrame({"week_end_dts":week_2019_during,"groups":"2019_during"})
df_2018_before=pd.DataFrame({"week_end_dts":week_2018_before,"groups":"2018_before"})
df_2018_during=pd.DataFrame({"week_end_dts":week_2018_during,"groups":"2018_during"})

df_all_week_end_dts=df_all_week_end_dts.append([df_2019_before,df_2019_during,df_2018_before,df_2018_during])

df_all_week_end_dts['week_end_dts']=df_all_week_end_dts['week_end_dts'].astype(str)
list_all_week_end_dts=df_all_week_end_dts['week_end_dts'].tolist()

In [3]:
len(list_all_week_end_dts)

24

In [4]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

In [5]:
sales_2018_subclass_1=glob.glob("/home/jian/BigLots/hist_daily_data_subclasslevel/*.txt")
sales_2018_subclass_1=[x for x in sales_2018_subclass_1 if x.split("MediaStormDailySales_week_ending_")[1][:10] in list_all_week_end_dts]

sales_2018_subclass_2=list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/"))
sales_2018_subclass_2=[x for x in sales_2018_subclass_2 if "aily" in x]
sales_2018_subclass_2=[x for x in sales_2018_subclass_2 if x.split("MediaStorm_")[1][:10] in list_all_week_end_dts]
sales_2018_subclass=sorted(sales_2018_subclass_1+sales_2018_subclass_2)
sales_2018_subclass=[x for x in sales_2018_subclass if ".txt" in x]

In [6]:
sales_2019_item=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
sales_2019_item=[x for x in sales_2019_item if "aily" in x]
sales_2019_item=[x for x in sales_2019_item if x.split("MediaStorm_")[1][:10] in list_all_week_end_dts]
sales_2019_item=sorted(sales_2019_item)
sales_2019_item=[x for x in sales_2019_item if ".txt" in x]


In [7]:
df_store_list_in_test=pd.read_excel("./Store_list_by_group.xlsx",dtype=str)
store_list_in_test=df_store_list_in_test['location_id'].tolist()

In [8]:
df_sales_2018_rewards=pd.DataFrame()
df_sales_2018_Nonrewards=pd.DataFrame()
df_sales_2018_Total=pd.DataFrame()

for file_2018 in sales_2018_subclass:
    df=pd.read_table(file_2018,nrows=samplerows,sep="|",dtype=str,
                     usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','subclass_transaction_amt'])
    df=pd.merge(df_store_list_in_test,df,on="location_id",how="left")
    df=df[pd.notnull(df['transaction_dt'])]
    df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
    week_start=df['transaction_dt'].min()
    week_end=df['transaction_dt'].max()
    
    df_total_sales=df.groupby(['location_id','Test_Groups'])['subclass_transaction_amt'].sum().to_frame().reset_index().rename(columns={"subclass_transaction_amt":"total_sales"})
    df_total_trans=df[['location_id','Test_Groups','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_total_trans=df_total_trans.groupby(['location_id','Test_Groups'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"total_trans"})
    df_total=pd.merge(df_total_sales,df_total_trans,on=['location_id','Test_Groups'],how="outer")
    df_total['week_start_dt']=week_start
    df_total['week_end_dt']=week_end
    
    
    df_rewards=df[pd.notnull(df['customer_id_hashed'])]
    df_rewards_sales=df_rewards.groupby(['location_id','Test_Groups'])['subclass_transaction_amt'].sum().to_frame().reset_index().rename(columns={"subclass_transaction_amt":"Rewards_sales"})
    df_rewards_trans=df_rewards[['location_id','Test_Groups','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_rewards_trans=df_rewards_trans.groupby(['location_id','Test_Groups'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"Rewards_trans"})
    df_rewards=pd.merge(df_rewards_sales,df_rewards_trans,on=['location_id','Test_Groups'],how="outer")
    df_rewards['week_start_dt']=week_start
    df_rewards['week_end_dt']=week_end
    
    
    df_NonRewards=df[pd.isnull(df['customer_id_hashed'])]
    df_Nonrewards_sales=df_NonRewards.groupby(['location_id','Test_Groups'])['subclass_transaction_amt'].sum().to_frame().reset_index().rename(columns={"subclass_transaction_amt":"Nonrewards_sales"})
    df_Nonrewards_trans=df_NonRewards[['location_id','Test_Groups','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_Nonrewards_trans=df_Nonrewards_trans.groupby(['location_id','Test_Groups'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"Nonrewards_trans"})
    df_Nonrewards=pd.merge(df_Nonrewards_sales,df_Nonrewards_trans,on=['location_id','Test_Groups'],how="outer")
    df_Nonrewards['week_start_dt']=week_start
    df_Nonrewards['week_end_dt']=week_end
    
    
    df_sales_2018_rewards=df_sales_2018_rewards.append(df_rewards)
    df_sales_2018_Nonrewards=df_sales_2018_Nonrewards.append(df_Nonrewards)
    df_sales_2018_Total=df_sales_2018_Total.append(df_total)
    print(datetime.datetime.now(),file_2018)

2019-08-05 13:35:44.488400 /home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-16/MediaStormDailySales.txt
2019-08-05 13:36:27.225292 /home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-23/MediaStormDailySales.txt
2019-08-05 13:37:44.028392 /home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-30/MediaStormDailySales.txt
2019-08-05 13:39:30.829771 /home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-07/MediaStormDailySales.txt
2019-08-05 13:40:33.686839 /home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-14/MediaStormDailySales20180717-113630-767.txt
2019-08-05 13:41:22.836278 /home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-21/MediaStormDailySales20180724-113327-741.txt
2019-08-05 13:42:08.860737 /home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-28/MediaStormDailySales20180731-111804-489.txt
2019-08-05 13:42:42.613401 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-05-12.txt
2019-08-05 13:43:25.215457 /home/jian/BigLots/hist_daily_data_subclass

In [9]:
df_sales_2019_rewards=pd.DataFrame()
df_sales_2019_Nonrewards=pd.DataFrame()
df_sales_2019_Total=pd.DataFrame()

for file_2019 in sales_2019_item:
    df=pd.read_table(file_2019,nrows=samplerows,sep="|",dtype=str,
                     usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','item_transaction_amt'])
    df=pd.merge(df_store_list_in_test,df,on="location_id",how="left")
    df=df[pd.notnull(df['transaction_dt'])]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    week_start=df['transaction_dt'].min()
    week_end=df['transaction_dt'].max()
    
    df_total_sales=df.groupby(['location_id','Test_Groups'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"total_sales"})
    df_total_trans=df[['location_id','Test_Groups','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_total_trans=df_total_trans.groupby(['location_id','Test_Groups'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"total_trans"})
    df_total=pd.merge(df_total_sales,df_total_trans,on=['location_id','Test_Groups'],how="outer")
    df_total['week_start_dt']=week_start
    df_total['week_end_dt']=week_end
    
    
    df_rewards=df[pd.notnull(df['customer_id_hashed'])]
    df_rewards_sales=df_rewards.groupby(['location_id','Test_Groups'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"Rewards_sales"})
    df_rewards_trans=df_rewards[['location_id','Test_Groups','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_rewards_trans=df_rewards_trans.groupby(['location_id','Test_Groups'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"Rewards_trans"})
    df_rewards=pd.merge(df_rewards_sales,df_rewards_trans,on=['location_id','Test_Groups'],how="outer")
    df_rewards['week_start_dt']=week_start
    df_rewards['week_end_dt']=week_end
    
    
    df_NonRewards=df[pd.isnull(df['customer_id_hashed'])]
    df_Nonrewards_sales=df_NonRewards.groupby(['location_id','Test_Groups'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"Nonrewards_sales"})
    df_Nonrewards_trans=df_NonRewards[['location_id','Test_Groups','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_Nonrewards_trans=df_Nonrewards_trans.groupby(['location_id','Test_Groups'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"Nonrewards_trans"})
    df_Nonrewards=pd.merge(df_Nonrewards_sales,df_Nonrewards_trans,on=['location_id','Test_Groups'],how="outer")
    df_Nonrewards['week_start_dt']=week_start
    df_Nonrewards['week_end_dt']=week_end
    
    
    df_sales_2019_rewards=df_sales_2019_rewards.append(df_rewards)
    df_sales_2019_Nonrewards=df_sales_2019_Nonrewards.append(df_Nonrewards)
    df_sales_2019_Total=df_sales_2019_Total.append(df_total)
    print(datetime.datetime.now(),file_2019)

2019-08-05 13:50:05.357710 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-11/MediaStormDailySales20190514-121223-405.txt
2019-08-05 13:50:48.765051 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-18/MediaStormDailySales20190521-111232-391.txt
2019-08-05 13:51:43.437578 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-25/MediaStormDailySales20190528-111211-863.txt
2019-08-05 13:53:30.611627 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-01/MediaStormDailySales20190604-111936-950.txt
2019-08-05 13:54:30.005347 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-08/MediaStormDailySales20190611-122013-442.txt
2019-08-05 13:55:35.553570 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-15/MediaStormDailySales20190618-112500-846.txt
2019-08-05 13:56:39.192299 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-22/MediaStormDailySales20190625-111927-257.txt
2019-08-05 13:57:33.252254 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-29/MediaStormDailySales20190702-113523-

In [27]:
def rename_cols_after_2_pivot(df):
    df=df.reset_index()
    new_cols=[]
    for col in df.columns.tolist():
        if col[1]=="":
            new_col=col[0]
        else:
            new_col=col[0]+" | "+col[1]
        new_cols=new_cols+[new_col]
    
    df.columns=new_cols
    return df


df_sales_2019_Total_wide_sales=pd.pivot_table(df_sales_2019_Total, values=['total_sales','total_trans'], index=['location_id', 'Test_Groups'],columns=['week_start_dt'])
df_sales_2019_Rewards_wide_sales=pd.pivot_table(df_sales_2019_rewards, values=['Rewards_sales','Rewards_trans'], index=['location_id', 'Test_Groups'],columns=['week_start_dt'])
df_sales_2019_NonRewards_wide_sales=pd.pivot_table(df_sales_2019_Nonrewards, values=['Nonrewards_sales','Nonrewards_trans'], index=['location_id', 'Test_Groups'],columns=['week_start_dt'])

df_sales_2018_Total_wide_sales=pd.pivot_table(df_sales_2018_Total, values=['total_sales','total_trans'], index=['location_id', 'Test_Groups'],columns=['week_start_dt'])
df_sales_2018_Rewards_wide_sales=pd.pivot_table(df_sales_2018_rewards, values=['Rewards_sales','Rewards_trans'], index=['location_id', 'Test_Groups'],columns=['week_start_dt'])
df_sales_2018_NonRewards_wide_sales=pd.pivot_table(df_sales_2018_Nonrewards, values=['Nonrewards_sales','Nonrewards_trans'], index=['location_id', 'Test_Groups'],columns=['week_start_dt'])

df_sales_2019_Total_wide_sales=rename_cols_after_2_pivot(df_sales_2019_Total_wide_sales)
df_sales_2019_Rewards_wide_sales=rename_cols_after_2_pivot(df_sales_2019_Rewards_wide_sales)
df_sales_2019_NonRewards_wide_sales=rename_cols_after_2_pivot(df_sales_2019_NonRewards_wide_sales)
df_sales_2018_Total_wide_sales=rename_cols_after_2_pivot(df_sales_2018_Total_wide_sales)
df_sales_2018_Rewards_wide_sales=rename_cols_after_2_pivot(df_sales_2018_Rewards_wide_sales)
df_sales_2018_NonRewards_wide_sales=rename_cols_after_2_pivot(df_sales_2018_NonRewards_wide_sales)



df_both_year_Total=pd.merge(df_sales_2019_Total_wide_sales,df_sales_2018_Total_wide_sales,on=['location_id', 'Test_Groups'],how="outer").sort_values(['Test_Groups','location_id'])
df_both_year_Rewards=pd.merge(df_sales_2019_Rewards_wide_sales,df_sales_2018_Rewards_wide_sales,on=['location_id', 'Test_Groups'],how="outer").sort_values(['Test_Groups','location_id'])
df_both_year_NonRewards=pd.merge(df_sales_2019_NonRewards_wide_sales,df_sales_2018_NonRewards_wide_sales,on=['location_id', 'Test_Groups'],how="outer").sort_values(['Test_Groups','location_id'])


In [35]:
df_both_year_Total_valid=df_both_year_Total[df_both_year_Total.notnull().all(axis=1)]
df_both_year_Total_exclude=df_both_year_Total[df_both_year_Total.isnull().any(axis=1)]

df_both_year_Rewards_valid=df_both_year_Rewards[df_both_year_Rewards.notnull().all(axis=1)]
df_both_year_Rewards_exclude=df_both_year_Rewards[df_both_year_Rewards.isnull().any(axis=1)]

df_both_year_NonRewards_valid=df_both_year_NonRewards[df_both_year_NonRewards.notnull().all(axis=1)]
df_both_year_NonRewards_exclude=df_both_year_NonRewards[df_both_year_NonRewards.isnull().any(axis=1)]

In [36]:
writer=pd.ExcelWriter("./BL_sales_for_stores_test2_test_3_yoy_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_both_year_Total_valid.to_excel(writer,"total_valid",index=False)
df_both_year_Rewards_valid.to_excel(writer,"Rewards_valid",index=False)
df_both_year_NonRewards_valid.to_excel(writer,"Nonrewards_valid",index=False)

df_both_year_Total_exclude.to_excel(writer,"total_excluded",index=False)
df_both_year_Rewards_exclude.to_excel(writer,"Rewards_excluded",index=False)
df_both_year_NonRewards_exclude.to_excel(writer,"Nonrewards_excluded",index=False)
writer.save()

In [37]:
df_both_year_NonRewards.shape

(82, 50)